armo un ELT en jupiter para probar el proceso y luego migrarlo al .py

importamos las librerias, puede que haya de más pero luego ya las iremos depurando

In [2]:
import os
import pandas as pd
from pandas.io import sql
import numpy as np
import chardet
from pathlib import Path as pl
import pyarrow
import mysql
import re
import json
import datetime


importamos las funciones que vamos creando en paralelo para el ETL

In [3]:
from funciones_etl import *

creamos las variables con las paths_ vamos a respetar el orden alfabético de los archivos durante el EDA y ETL

In [4]:
path_amazon="Datasets\\amazon_prime_titles.csv"
path_disney="Datasets\\disney_plus_titles.csv"
path_hulu="Datasets\\hulu_titles.csv"
path_netflix="Datasets\\netflix_titles.json"

Se abren los archivos con la función de apertura creada y luego arrancamos el EDA


In [5]:
df_amazon=abriendo_archivos(path_amazon)
df_disney=abriendo_archivos(path_disney)
df_hulu=abriendo_archivos(path_hulu)
df_netflix=abriendo_archivos(path_netflix)

DESARROLLO DEL EDA:
- 1° -  se abrieron los archivos csv en note pad++ para ver que tipo de separación tenía y encoding 
con visual studio code, con pandas y python: 
- 2° - se corre un sample(20) varias veces cada uno para descrubrir que tipos de datos hay
- 3° - con ,info se analizan cuantos registros y cuantas columnas tienen
- 4° - se analizan los valores faltantes
- 5° - se analizan los duplicados de de los IP (count. distinct de la columna)
- 6° - aqui se debería realizar un informe al cliente y tomar decisiones sobre la estrategia de ETL


In [6]:
#"s1896" in df_amazon.show_id.values

analisis global con .sample, se ven los nombres de las columnas, puede iterarse y se ve el general de que tipo de datos se trata

In [19]:
df_amazon.sample(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
4754,s4755,Movie,The Crystal Ball,Elliott Nugent,"Ray Milland, Paulette Goddard, Gladys George",NaN,NaN,1943,13+,81 min,Comedy,Toni Gerard lands in New York with 38 cents to...
8258,s8259,Movie,Stuart Little,Rob Minkoff,"Geena Davis, Michael J. Fox, Hugh Laurie, Jona...",NaN,NaN,1999,PG,85 min,"Action, Kids",E.B. White's family classic about the mouse wh...
7698,s7699,Movie,The Habit of Beauty,Mirko Pincelli,"Vincenzo Amato, Francesca Neri, Noel Clarke, N...",NaN,NaN,2016,13+,89 min,"Drama, Romance",A successful professional couple is torn apart...
6450,s6451,Movie,Purgatory,Ami Artzi,"Tanya Roberts, Julie Pop, Hal Orlandini, Adrie...",NaN,NaN,1988,18+,93 min,"Action, Drama",Two female Peace Corps volunteers in Africa ar...
2417,s2418,Movie,Frontier Pony Express,Joseph Kane,"Roy Rogers, Lynne Roberts, Raymond Hatton, Edw...",NaN,NaN,1939,18+,53 min,"Military and War, Western",Roy Rogers plays a pony express rider who trav...
3858,s3859,Movie,Gully Boy,Zoya Akhtar,"Ranveer Singh, Alia Bhatt, Kalki Koechlin, Sid...",NaN,NaN,2019,16+,154 min,"Drama, International",A coming-of-age story based on the lives of st...
7595,s7596,Movie,The Waste Land,WANG MENG,"WANG MENG, Jiao Zhonghui",NaN,NaN,2021,16+,130 min,Drama,A rural youth who has nothing to do and become...
5571,s5572,Movie,"Little Baby Bum - Learn Colors, Shapes and Num...",Cannis Holder,"Grace Tamayo, Erin Webbs",NaN,NaN,2020,ALL,36 min,Kids,"Little Baby Bum is a fun, educational children..."
4640,s4641,Movie,Bhoomi,Omung Kumar,"Sanjay Dutt, Aditi Rao Hydari, Sharad Kelkar, ...",India,NaN,2017,13+,133 min,"Action, Drama, International",A revenge drama that centers on the relationsh...
6772,s6773,Movie,Marital Problems,Dia Taylor,"Nick Capper, Callum Gault, Aleis Duffy, Neil G...",NaN,NaN,2017,16+,94 min,Comedy,"On the eve of his eviction, Ian's home becomes..."


In [8]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [9]:
df_amazon.isnull().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

In [20]:
df_disney.sample(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
773,s774,Movie,Can of Worms,Paul Schneider,"Michael Shulman, Erika Christensen, Adam Wylie...","United States, Canada","November 12, 2019",1999,TV-PG,84 min,"Comedy, Coming of Age, Science Fiction",Aliens invade when a teen asks for intergalact...
587,s588,Movie,Sea Scouts,Dick Lundy,NaN,United States,"April 3, 2020",1939,TV-PG,8 min,"Animation, Family",Donald and his nephews encounter a shark. See ...
879,s880,TV Show,Encore!,NaN,"Kristen Bell, Roger Love, Susan Egan, Ali Stro...",United States,"November 12, 2019",2019,TV-14,1 Season,"Music, Reality",Kristen Bell reunites former high school class...
963,s964,Movie,Inspector Gadget 2,Alex Zamm,"French Stewart, Elaine Hendrix, Caitlin Wachs,...",United States,"November 12, 2019",2003,G,90 min,"Action-Adventure, Comedy, Family","While Inspector Gadget is on probation, Claw b..."
303,s304,Movie,You Again,Andy Fickman,"Kristen Bell, Sigourney Weaver, Jamie Lee Curt...",United States,"February 12, 2021",2010,PG,106 min,Comedy,An unlikely reunion causes high school drama t...
1023,s1024,Movie,Lonesome Ghosts,Burt Gillett,NaN,United States,"November 12, 2019",1937,TV-Y7,9 min,"Animation, Family, Kids","Mickey Mouse, Donald Duck and Goofy run a ghos..."
1067,s1068,Movie,Meet the Robinsons,Stephen Anderson,"Angela Bassett, Daniel Hansen, Jordan Fry, Mat...",United States,"November 12, 2019",2007,G,97 min,"Action-Adventure, Animation, Family",A young inventor uses time travel to find the ...
1109,s1110,Movie,Oliver & Company,George Scribner,"Joey Lawrence, Billy Joel, Cheech Marin, Richa...",United States,"November 12, 2019",1988,G,76 min,"Action-Adventure, Animation, Musical",A feisty young cat explores New York in this a...
780,s781,Movie,Cars Toon: Heavy Metal Mater,John Lasseter,"Larry the Cable Guy , Keith Ferguson",United States,"November 12, 2019",2010,TV-G,5 min,"Animation, Comedy, Family",Mater is a rock star in a heavy metal band.
987,s988,TV Show,Kickin' It,NaN,"Leo Howard, Dylan Snyder, Mateo Arias, Olivia ...",United States,"November 12, 2019",2010,TV-PG,4 Seasons,"Action-Adventure, Comedy, Coming of Age",The series will follow the crew of lovable mis...


In [11]:
df_disney.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [12]:
df_disney.isnull().sum()

show_id           0
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
dtype: int64

In [13]:
df_hulu.sample(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1846,s1847,TV Show,Magi 2,NaN,NaN,NaN,"January 31, 2020",2012,TV-14,1 Season,"Action, Adventure, Anime","A young boy, Aladin, travels a mysterious cont..."
134,s135,TV Show,Station 19,NaN,NaN,United States,"October 1, 2021",2018,TV-14,5 Seasons,"Action, Drama","In Seattle, all firefighters are also trained ..."
2760,s2761,TV Show,Mobile Suit Gundam Wing,NaN,NaN,"Japan, United States","February 20, 2017",1995,TV-MA,1 Season,"Anime, International, Science Fiction",The Revolution Has Begun! Mankind has moved in...
2025,s2026,Movie,Wounds,NaN,NaN,"United States, United Kingdom","October 18, 2019",2019,R,95 min,"Horror, Thriller",Disturbing and mysterious things begin happeni...
1998,s1999,Movie,Framing John DeLorean,NaN,NaN,United States,"November 5, 2019",2019,109 min,NaN,Documentaries,"Money, power, politics, drugs, scandal, and fa..."
1579,s1580,Movie,Buffaloed,NaN,NaN,United States,"June 18, 2020",2019,95 min,NaN,Comedy,A woman is desperate to escape her life in Buf...
1954,s1955,Movie,ID: INVADED,NaN,NaN,Japan,"December 15, 2019",2019,1 Season,NaN,"Anime, Crime, Drama","Meet Sakaido, a brilliant detective who solves..."
2772,s2773,TV Show,Chowder,NaN,NaN,"United States, China","January 1, 2017",2014,TV-Y7,3 Seasons,"Cartoons, Comedy, Kids",Chowder follows an aspiring young chef named C...
2109,s2110,Movie,The Amazing Johnathan Documentary,NaN,NaN,United States,"August 16, 2019",2019,TV-MA,91 min,Documentaries,What begins as a seemingly ordinary profile of...
535,s536,Movie,Personal Shopper,NaN,NaN,NaN,"July 1, 2021",2016,R,105 min,"Drama, Horror, Mystery",A young psychic is plunged into a dangerous ga...


In [14]:
df_hulu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [15]:
df_hulu.isnull().sum()

show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
dtype: int64

In [16]:
df_netflix.sample(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
4862,s4863,TV Show,Saint Seiya: The Lost Canvas,None,"Tetsuya Kakihara, Hiro Shimono, Aya Hirano, At...",Japan,"May 15, 2018",2009,TV-MA,1 Season,"Anime Series, International TV Shows",This anime adventure follows the battle betwee...
8702,s8703,Movie,Waterschool,Tiffanie Hsu,None,"United States, Brazil, India, Uganda, China","July 25, 2018",2018,TV-G,68 min,Documentaries,"Six young women, who each live near one of Ear..."
3011,s3012,TV Show,Hip-Hop Evolution,None,Shad Kabango,Canada,"January 17, 2020",2020,TV-MA,4 Seasons,"Docuseries, International TV Shows","Interviews with influential MCs, DJs and mogul..."
3787,s3788,TV Show,Bad Blood,None,"Anthony LaPaglia, Kim Coates, Enrico Colantoni...",Canada,"May 31, 2019",2019,TV-MA,2 Seasons,"Crime TV Shows, TV Dramas",This sprawling crime drama follows the true st...
624,s625,Movie,Prime Time,Jakub Piątek,"Bartosz Bielenia, Magdalena Popławska, Andrzej...",None,"June 30, 2021",2021,TV-MA,92 min,"Dramas, International Movies, Thrillers","On New Year’s Eve 1999, an armed man enters a ..."
8211,s8212,Movie,The Black Room,Rolfe Kanefsky,"Natasha Henstridge, Lukas Hassel, Lin Shaye, D...",United States,"August 7, 2017",2016,TV-MA,95 min,Horror Movies,A couple's new dream home morphs into a nightm...
226,s227,TV Show,Post Mortem: No One Dies in Skarnes,None,"Kathrine Thorborg Johansen, Elias Holmen Søren...",None,"August 25, 2021",2021,TV-MA,1 Season,"International TV Shows, TV Comedies, TV Dramas",She's back from the dead and has a newfound th...
6161,s6162,TV Show,Ancient Aliens,None,Robert Clotworthy,United States,"September 17, 2019",2018,TV-14,1 Season,"Docuseries, Science & Nature TV",Were ancient humans really behind some of the ...
2022,s2023,TV Show,Julie and the Phantoms,Kenny Ortega,"Madison Reyes, Charlie Gillespie, Owen Patrick...",United States,"September 10, 2020",2020,TV-G,1 Season,"Kids' TV, TV Comedies",Julie lost her passion for music when she lost...
6200,s6201,Movie,Avengers: Infinity War,"Anthony Russo, Joe Russo","Robert Downey Jr., Josh Brolin, Mark Ruffalo, ...",United States,"December 25, 2018",2018,PG-13,150 min,"Action & Adventure, Sci-Fi & Fantasy",Superheroes amass to stop intergalactic sociop...


In [17]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 894.5+ KB


In [18]:
df_netflix.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

Se continua analizando en detalle distintas columnas

Conclusiones del EDA: 

GENERALES (considerando el objetivo y alcance de este proyecto): 

    - Las columnas duration tienen tanto información en minutos como de temporadas

    - Las columnas cast tienen los actores separados por una coma

PARTICULARES

- 1- amazon_prime_titles.csv
    - separdor ','+ encoding utf-8
    - cararcteres especiales [ (30) 
    - valores faltantes 
        - director        2082
        - cast            1233
        - country         8996
        - date_added      9513
        - rating           337
    - 9687 registros 
    - 12 columnas

- 2- disney_plus_titles.csv
    - separdor ','+ encoding utf-8
    - cararcteres especiales [ (NO)  
    - valores faltantes 
        - director        473
        - cast            190
        - country         219
        - date_added        3
        - rating            3
    - 1450 registros 
    - 12 columnas

- 3- hulu_titles.csv
    - separdor ','+ encoding utf-8
    - cararcteres especiales [ (1)  
    - valores faltantes 
        - director        3070
        - cast            3073
        - country         1453
        - date_added        28
        - rating           520
        - duration         479
        - description        4
    - 3090 registros 
    - 12 columnas

- 4- netflix_titles.json
    - valores faltantes 
        - director        2634
        - cast             825
        - country          831
        - date_added        10
        - rating             4
        - duration           3
    - 8807 registros
    - 12 columnas